In [51]:
!pip3 install datawig

Processing c:\users\rupal\appdata\local\pip\cache\wheels\9b\a2\80\f3c44c1e8357a5ef3e85a0fd7a323511266ec42762d91e1025\datawig-0.1.12-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4a/bd/eee1157fc2d8514970b345d69cb9975dcd1e42cd7e61146ed841f6e68309/typing-3.6.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f0/ac/92c3d2f0b627efbd1a7b2156faa697f9c2bbd7b0fe83ba8a9d36f982156f/pandas-0.25.3-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/d0/19/6d56792b95445fc9b818e8d8eff3d5fff1221c40fe06a096c9587f2cb1d3/mxnet-1.4.0-py2.py3-none-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/53/39/4ab213673844e0c004bed8a0781a0721a3f6bb23eb8854ee75c236428892/graphviz-0.8.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/c

ERROR: py2neo 4.1.0 has requirement prompt-toolkit==1.0.15, but you'll have prompt-toolkit 2.0.7 which is incompatible.
ERROR: mxnet 1.4.0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'd:\\users\\rupal\\anaconda3\\lib\\site-packages\\~andas\\io\\msgpack\\_packer.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [53]:
import pandas as pd
import numpy as np

from jenga.tasks.income import IncomeEstimationTask
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

import datawig

In [24]:
dat = IncomeEstimationTask(seed=10)

In [25]:
train_dat = dat.train_data
train_lab = dat.train_labels

In [26]:
print(type(train_dat))
print(type(train_lab))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [27]:
train_dat.head()

,workclass,occupation,marital_status,education,hours_per_week,age
190,Private,Machine-op-inspct,Divorced,Some-college,28,38
30924,Private,Other-service,Never-married,7th-8th,40,39
4802,Private,Machine-op-inspct,Married-civ-spouse,HS-grad,40,25
22918,Private,Machine-op-inspct,Widowed,HS-grad,40,60
10198,Private,Craft-repair,Married-civ-spouse,HS-grad,40,43


In [64]:
train_dat.isnull().sum()

workclass         1453
occupation        1460
marital_status       0
education            0
hours_per_week       0
age                  0
dtype: int64

In [28]:
train_lab[0:5]

array([False, False, False, False,  True])

In [29]:
test_dat = dat.test_data

In [30]:
# fit a baseline model
model = dat.fit_baseline_model(train_dat, train_lab)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.2min finished
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Sp

In [31]:
# get prediction probability estimates
pred_prob = model.predict_proba(test_dat)
dat.score_on_test_data(pred_prob)

D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


0.8835320908097553

#### Test on correupted data

In [47]:
# corrupt the test data
missing_values_corruption = MissingValues(column="age", fraction=0.45, missingness="MAR", na_value=999)

corrupt_test_dat = missing_values_corruption.transform(test_dat)
corrupt_test_dat

,workclass,occupation,marital_status,education,hours_per_week,age
26952,Self-emp-not-inc,Sales,Married-civ-spouse,7th-8th,40,999
19336,State-gov,Adm-clerical,Separated,HS-grad,38,46
30953,Private,Machine-op-inspct,Never-married,HS-grad,40,34
27069,State-gov,Prof-specialty,Never-married,Bachelors,50,27
14911,NaN,NaN,Married-civ-spouse,7th-8th,40,999
2879,Private,Other-service,Never-married,11th,40,22
14586,Local-gov,Exec-managerial,Married-civ-spouse,Bachelors,40,999
9834,Private,Machine-op-inspct,Never-married,HS-grad,40,29
29997,Local-gov,Adm-clerical,Married-civ-spouse,Some-college,35,999
10833,Private,Machine-op-inspct,Never-married,HS-grad,40,21


In [48]:
# get the prediction probability estimates on the corrupted dataset
pred_prob_currupt = model.predict_proba(corrupt_test_dat)
dat.score_on_test_data(pred_prob_currupt)

D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


0.857333640570936

In [56]:
# corruption impact evaluator
evaluator = CorruptionImpactEvaluator(dat)

num_repetitions = 10
results = evaluator.evaluate(model, num_repetitions, missing_values_corruption)

D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


0/10 (0.09375)


D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)
D:\Users\rupal\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transfor

In [57]:
results

In [59]:
print(f"""
Score (AUC) on
    data: {results[0].baseline_score}
    corrupted data: {np.mean(results[0].corrupted_scores)}
""")


Score (AUC) on
    data: 0.8835320908097553
    corrupted data: 0.7104483275279104



#### Missing value imputer for the columns with null values

In [66]:
## using this part from the cikm...mis... notebook

class DataWigImputer:
    def __init__(self, input_cols, target_col):
        self.input_cols = input_cols
        self.target_col = target_col
        self.model = None
    
    def fit(self, data):
        self.model = datawig.SimpleImputer(
            input_columns = self.input_cols,
            output_column = self.target_col,
            output_path = f'imputer_model_{self.target_col}',
            is_explainable = False)
        
        self.model = self.model.load(f'imputer_model_{self.target_col}')
        
    def transform(self, data):
        imputed = data
        imputed = self.model.predict(imputed, inplace = True)
        imputed.loc[imputed[self.target_col].isnull(), self.target_col] = imputed[self.target_col + '_imputed']
        
        return imputed

#### Scenarios from the CleanML paper